<a href="https://colab.research.google.com/github/Shuaib11-Github/ALMABETTER/blob/main/Fine_Tuning_phi_2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate einops tqdm scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 910.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
import torch
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
AutoModelForCausalLM,
AutoTokenizer,
BitsAndBytesConfig,
HfArgumentParser,
TrainingArguments
)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
from tqdm.notebook import tqdm
from trl import SFTTrainer

In [ ]:
from huggingface_hub import interpreter_login

In [ ]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
dataset = load_dataset("Amod/mental_health_counseling_conversations", split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)

In [ ]:
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3512 entries, 0 to 3511
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Context   3512 non-null   object
 1   Response  3512 non-null   object
dtypes: object(2)
memory usage: 55.0+ KB


In [ ]:
def format_row(row):

  question = row["Context"]
  answer = row["Response"]
  formatted_string = f"[INST] {question} [/INST] {answer}"

  return formatted_string

In [ ]:
df["Text"] = df.apply(format_row, axis = 1)

In [ ]:
df['Text']

0       [INST] I'm going through some things with my f...
1       [INST] I'm going through some things with my f...
2       [INST] I'm going through some things with my f...
3       [INST] I'm going through some things with my f...
4       [INST] I'm going through some things with my f...
                              ...                        
3507    [INST] My grandson's step-mother sends him to ...
3508    [INST] My boyfriend is in recovery from drug a...
3509    [INST] The birth mother attempted suicide seve...
3510    [INST] I think adult life is making him depres...
3511    [INST] I just took a job that requires me to t...
Name: Text, Length: 3512, dtype: object

In [ ]:
df

,Context,Response,Text
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",[INST] I'm going through some things with my f...
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",[INST] I'm going through some things with my f...
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,[INST] I'm going through some things with my f...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,[INST] I'm going through some things with my f...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,[INST] I'm going through some things with my f...
...,...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...,[INST] My grandson's step-mother sends him to ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...,[INST] My boyfriend is in recovery from drug a...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit...",[INST] The birth mother attempted suicide seve...
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...,[INST] I think adult life is making him depres...


In [ ]:
new_df = df[['Text']]

In [ ]:
new_df.head()

,Text
0,[INST] I'm going through some things with my f...
1,[INST] I'm going through some things with my f...
2,[INST] I'm going through some things with my f...
3,[INST] I'm going through some things with my f...
4,[INST] I'm going through some things with my f...


In [ ]:
new_df.to_csv('formatted_data.csv', index = False)

In [ ]:
training_dataset = load_dataset("csv", data_files = "formatted_data.csv", split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
training_dataset

Dataset({
    features: ['Text'],
    num_rows: 3512
})

### training_arguments = TrainingArguments(
    output_dir = "./mhGPT",
    num_train_epochs = 2,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 12,
    evaluation_strategy = "steps",
    eval_steps = 1500,
    logging_steps = 15,
    optim = "paged_adamw_8bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "cosine",
    save_steps = 1500,
    warmup_ratio = 0.05,
    weight_decay = 0.01,
    max_steps = -1
)

## Fine Tuning

In [ ]:
base_model = "microsoft/phi-2"
new_model = "phi-2_mental_health"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig (
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float64,
    bnb_4bit_use_double_quant = False
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    trust_remote_code = True,
    flash_attn = True,
    flash_rotary = True,
    fused_dense = True,
    low_cpu_mem_usage = True,
    device_map = {"": 0},
    revision = "refs/pr/23"
)

model.config.use_cache = False
model.config.pre_training_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = True)


training_arguments = TrainingArguments(
    output_dir = "./mhGPT",
    evaluation_strategy="steps",
    # do_eval=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=12,
    per_device_eval_batch_size=1,
    log_level="debug",
    save_steps=100,
    logging_steps=25,
    learning_rate=1e-4,
    eval_steps=50,
    optim='paged_adamw_8bit',
    fp16=True, #change to bf16 if are using an Ampere GPU
    num_train_epochs=2,
    warmup_steps=100,
    lr_scheduler_type="linear",
)

peft_config = LoraConfig(
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
    # target_modules = ["Wqkv", "fc1", "fc2"],
    target_modules = ["Wqkv", "out_proj"]
)

trainer = SFTTrainer(
    model = model,
    train_dataset = training_dataset,
    peft_config = peft_config,
    dataset_text_field = "Text",
    max_seq_length = 690,
    tokenizer = tokenizer,
    args = training_arguments,
    packing=True
)

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/834565c23f9b28b96ccbeabe614dd906b6db551a/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/834565c23f9b28b96ccbeabe614dd906b6db551a/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/834565c23f9b28b96ccbeabe614dd906b6db551a/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/834565c23f9b28b96ccbeabe614dd906b6db551a/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/834565c23f9b28b96ccbeabe614dd906b6db551a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/834565c23f9b28b96ccbeabe614dd906b6db551a/tokenizer_config.json
loadin

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing PhiForCausalLM.

All the weights of PhiForCausalLM were initialized from the model checkpoint at microsoft/phi-2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use PhiForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/78c707edeb2428936635019e94f568c1885ecc09/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "transformers_version": "4.31.0"
}

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will c

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Currently training with a batch size of: 1
***** Running training *****
  Num examples = 3,512
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 12
  Total optimization steps = 584
  Number of trainable parameters = 7,864,320
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
